# Tổng hợp Kết quả Nghiên cứu - 10 Mô hình Baseline
### Pipeline: Tự động đọc dữ liệu từ thư mục `Result/` và trực quan hóa so sánh.

**Mục tiêu:**
- Đọc file `report.json` của từng mô hình.
- So sánh các chỉ số Classification (Accuracy, F1-Score).
- So sánh các chỉ số Regression (MSE, RMSE, R2 Score).
- Trực quan hóa bằng biểu đồ bar chart và heatmap tổng hợp.

In [ ]:
import os, json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Cấu hình đường dẫn
BASE_PATH = r'd:\HUTECH\AI\DeepLearning\DAHS\MangoLeaf'
RESULT_PATH = os.path.join(BASE_PATH, 'Result')

def load_results(path):
    all_data = []
    if not os.path.exists(path):
        print(f'LỖI: Thư mục {path} không tồn tại!')
        return None
    
    for m_name in os.listdir(path):
        r_file = os.path.join(path, m_name, 'report.json')
        if os.path.exists(r_file):
            with open(r_file, 'r') as f:
                try:
                    data = json.load(f)
                    reg = data.get('regression_metrics', {})
                    all_data.append({
                        'Model': m_name,
                        'Accuracy': reg.get('accuracy', 0),
                        'MSE': reg.get('mse', 0),
                        'RMSE': reg.get('rmse', 0),
                        'R2_Score': reg.get('r2_score', 0),
                        'Macro_F1': data.get('macro avg', {}).get('f1-score', 0)
                    })
                except:
                    continue
    return pd.DataFrame(all_data)

df = load_results(RESULT_PATH)

if df is not None and not df.empty:
    df = df.sort_values(by='Accuracy', ascending=False)
    print('>>> Đã tải kết quả của', len(df), 'mô hình.')
    display(df.style.background_gradient(subset=['Accuracy', 'R2_Score'], cmap='viridis'))
else:
    print('Chưa có dữ liệu trong thư mục Result! Vui lòng chạy các cell training trước.')

### 1. So sánh các chỉ số Classification (Accuracy & F1-Score)

In [ ]:
if df is not None and not df.empty:
    plt.figure(figsize=(15, 6))
    
    # Bar chart cho Accuracy
    plt.subplot(1, 2, 1)
    sns.barplot(data=df, x='Model', y='Accuracy', palette='Blues_r')
    plt.title('So sánh Accuracy các mô hình', fontsize=14)
    plt.xticks(rotation=45)
    for i, v in enumerate(df['Accuracy']): 
        plt.text(i, v + 0.01, f'{v:.2f}', ha='center')
    
    # Bar chart cho Macro F1
    plt.subplot(1, 2, 2)
    sns.barplot(data=df, x='Model', y='Macro_F1', palette='Greens_r')
    plt.title('So sánh Macro F1-Score', fontsize=14)
    plt.xticks(rotation=45)
    for i, v in enumerate(df['Macro_F1']): 
        plt.text(i, v + 0.01, f'{v:.2f}', ha='center')
        
    plt.tight_layout()
    plt.show()

### 2. So sánh các chỉ số Regression (MSE, RMSE, R2 Score)
*Lưu ý: MSE thấp hơn là tốt hơn, R2 cao hơn là tốt hơn.*

In [ ]:
if df is not None and not df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # MSE Comparison
    sns.barplot(data=df.sort_values('MSE'), x='Model', y='MSE', ax=axes[0], palette='Reds')
    axes[0].set_title('So sánh Mean Squared Error (MSE)', fontsize=14)
    axes[0].tick_params(axis='x', rotation=45)
    
    # R2 Score Comparison
    sns.barplot(data=df, x='Model', y='R2_Score', ax=axes[1], palette='Purples_r')
    axes[1].set_title('So sánh R2 Score', fontsize=14)
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()